# Nipah Virus Inhibitor Research: Final Verification
**Student:** Vihaan Agrawal  
**Project:** Computational Discovery of Resistance-Proof Antivirals

## What is this notebook?
This is the "lab bench" where I actually run the physics simulations. I wrote this code to connect different tools together:
1.  **AutoDock Vina:** The physics engine (calculates energy).
2.  **PyMOL/Python:** The geometry engine (calculates distances).
3.  **Matplotlib:** The graphing tool.

---

### Step 1: Loading the Libraries
We need standard Python tools to run system commands and do math.

In [ ]:
import os
import subprocess  # Allows us to run terminal commands (like Vina) from Python
import numpy as np # Math library for calculating 3D distances
import matplotlib.pyplot as plt # Graphing library
from pathlib import Path

# Setting up paths so the code knows where files are
BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "data"
BIN_DIR = BASE_DIR / "bin"
VINA_PATH = BIN_DIR / "vina"

### Step 2: The Physics Engine (Docking)
This function is the core of the project. It acts like a "Cost Function Minimizer". 

Think of it like validitating a password, but instead of "Correct/Incorrect", it returns an **Energy Score** (in kcal/mol). 
*   **Lower is Better** (more negative = tighter binding).
*   -6.0 is weak.
*   -9.0 is very strong.

We use **Vinardo** scoring because it's better at handling oil-like (hydrophobic) interactions.

In [ ]:
def run_docking_simulation(ligand_name, receptor_file, ligand_file):
    """
    Runs the docking simulation. 
    - ligand_name: Just a label (e.g., 'BMS-986205')
    - receptor_file: The 3D model of the protein (WT or Mutant)
    - ligand_file: The 3D model of the drug
    """
    
    output_file = f"results_{ligand_name}.pdbqt"
    
    # This is the exact command we would type in the terminal
    # We are asking Vina to look inside a specific box (x=10, y=10, z=10)
    # exhaustiveness=16 means "try 16 times harder than default"
    cmd = [
        str(VINA_PATH),
        "--receptor", str(receptor_file),
        "--ligand", str(ligand_file),
        "--center_x", "10.5", "--center_y", "-4.2", "--center_z", "28.6",
        "--size_x", "22", "--size_y", "22", "--size_z", "22",
        "--scoring", "vinardo",
        "--exhaustiveness", "16",
        "--out", output_file
    ]
    
    # Run the command silently
    subprocess.run(cmd, capture_output=True, check=True)
    
    # Read the file to find the best score
    # Vina results look like: "REMARK VINA RESULT: -7.5 0.0 0.0"
    with open(output_file) as f:
        for line in f:
            if "VINA RESULT" in line:
                # Extract the first number (the best energy)
                score = float(line.split()[3])
                return score, output_file
    
    return 0.0, None

### Step 3: Collision Detection (The "Ghost Clash")
This is a custom safety check I wrote. 

When the virus mutates, it deletes a big atom (Tryptophan) and replaces it with a small one (Alanine). 
Bad drugs try to fill that new empty space. But if they do that, they won't fit in the original virus anymore.

This function checks 3D distance. If the drug is too close to where the Tryptophan *used* to be, we fail it.

In [ ]:
def check_ghost_clash(docked_file):
    """
    Checks if the drug is cheating by occupying the 'Ghost' space.
    Returns: True if SAFE, False if CLASHING.
    """
    # The exact 3D coordinates of the deleted Tryptophan side-chain
    # I got these from the original PDB file (9KNZ)
    GHOST_COORDS = np.array([12.5, -5.2, 29.1]) 
    
    min_dist = 999.0
    
    # Parse the PDBQT file to get every atom's x,y,z
    with open(docked_file) as f:
        for line in f:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                # Extracts x, y, z columns from the text file
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                atom_pos = np.array([x, y, z])
                
                # Euclidean distance formula: sqrt((x2-x1)^2 + ...)
                d = np.linalg.norm(atom_pos - GHOST_COORDS)
                if d < min_dist:
                    min_dist = d
    
    print(f"  > Distance to Ghost Site: {min_dist:.2f} Å")
    
    # If it's closer than 1.5 Angstroms, it's physically crashing
    if min_dist < 1.5:
        return False # Fail
    return True # Pass

### Step 4: Running the Experiment
Now I run the actual verification loop for two drugs:
1.  **BMS-986205:** My proposed Cure.
2.  **ERDRP-0519:** The Control (known to fail).

We test both against:
*   **Wild Type (WT):** The normal virus.
*   **Mutant (W730A):** The resistant virus.

In [ ]:
drugs = [
    {"name": "BMS-986205", "file": "data/ligand_BMS_986205.pdbqt"},
    {"name": "ERDRP-0519", "file": "data/ligand_ERDRP_0519.pdbqt"}
]

results = {}

print("=== Starting Research Verification ===")

for drug in drugs:
    name = drug["name"]
    print(f"\nTesting Candidate: {name}...")
    
    # 1. Dock against Wild Type (Standard Nipah)
    wt_score, _ = run_docking_simulation(name+"_WT", "data/receptor_wt.pdbqt", drug["file"])
    print(f"  > WT Affinity: {wt_score} kcal/mol")
    
    # 2. Dock against Mutant (Resistant Nipah)
    mut_score, mut_pose = run_docking_simulation(name+"_MUT", "data/receptor_mut.pdbqt", drug["file"])
    print(f"  > Mutant Affinity: {mut_score} kcal/mol")
    
    # 3. Check for Geometry Failures (Ghost Clash)
    is_safe = check_ghost_clash(mut_pose)
    if not is_safe:
        print("  > FAIL: Ghost Clash Detected! (Drug is fake-binding)")
        # Penalize the score significantly so it fails the graph
        mut_score = 0.0 
    
    # 4. Calculate the Delta (Resistance Gap)
    delta = abs(wt_score - mut_score)
    print(f"  > Resistance Delta: {delta:.2f}")
    
    results[name] = {"WT": wt_score, "Mutant": mut_score}

### Step 5: Visualizing Results
If **Delta is small**, the bars should be the same height. This means the drug works equally well on both.
If **Delta is large**, the Mutant bar will drop. This means the drug failed.

In [ ]:
labels = list(results.keys())
wt_scores = [results[d]["WT"] for d in labels]
mut_scores = [results[d]["Mutant"] for d in labels]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(8, 5))
rects1 = ax.bar(x - width/2, wt_scores, width, label='Wild Type (Normal)', color='#4287f5')
rects2 = ax.bar(x + width/2, mut_scores, width, label='Mutant (Resistant)', color='#f54242')

ax.set_ylabel('Binding Affinity (kcal/mol) - More Negative is Better')
ax.set_title('Resistance Profile: BMS-986205 vs Control')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Add labels on top of bars
ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

plt.ylim(-9, 0) # Flip y-axis because negative energy is good
plt.show()